In [16]:
# staging.ipynb
# ----------------------
# 1. Подключение к PostgreSQL
# ----------------------
import pandas as pd
from torgstat.db import get_engine, DB_SCHEMA
import sys
from sqlalchemy import text

try:
    # Создаём движок SQLAlchemy
    engine = get_engine()
    
    # Проверяем реальное подключение с использованием text()
    with engine.connect() as conn:
        result = conn.execute(text("SELECT 1 as test_connection"))
        test_result = result.scalar()
        
    if test_result == 1:
        print("✓ Подключение к PostgreSQL успешно установлено и проверено")
    else:
        print("✗ Ошибка: Не удалось проверить подключение")
        sys.exit(1)
        
except Exception as e:
    print(f"✗ Ошибка подключения к PostgreSQL: {e}")
    sys.exit(1)

# Функция для чтения таблицы из схемы analytics
def read_table(table_name: str):
    try:
        query = f'SELECT * FROM {DB_SCHEMA}.{table_name};'
        df = pd.read_sql(query, engine)
        print(f"✓ Таблица {table_name} успешно загружена ({len(df)} строк)")
        return df
    except Exception as e:
        print(f"✗ Ошибка загрузки таблицы {table_name}: {e}")
        raise

print("✓ Все проверки пройдены успешно")

✓ Подключение к PostgreSQL успешно установлено и проверено
✓ Все проверки пройдены успешно


In [17]:
# ----------------------
# 2. Читаем сырые таблицы
# ----------------------

df_users = read_table("users")
df_sessions = read_table("sessions")
df_subs = read_table("subscriptions")
df_invoices = read_table("invoices")
df_plans = read_table("plans")
df_events = read_table("events")

# Быстрый просмотр
print("Users:", df_users.shape)
print("Sessions:", df_sessions.shape)
print("Subscriptions:", df_subs.shape)
print("Invoices:", df_invoices.shape)
print("Plans:", df_plans.shape)
print("Events:", df_events.shape)

# Размер данных
print(f"Размер DataFrame: {type(df_users.shape)}")
print(f"Количество строк: {df_users.shape[0]}")
print(f"Количество столбцов: {df_users.shape[1]}")

# Общая информация
df_users.info()
df_sessions.info()
df_subs.info()
df_invoices.info()
df_plans.info()
df_events.info()


# Просмотр первых и случайных строк

display(df_users.head())
display(df_sessions.head())
display(df_subs.head())
display(df_invoices.head())
display(df_plans.head())
display(df_events.head())

# Проверка на пустые таблицы (дополнительная ячейка)
print("ПРОВЕРКА НА ПУСТЫЕ ТАБЛИЦЫ:")
print("═" * 50)
for name, df in zip(["users", "sessions", "subscriptions", "invoices", "plans", "events"],
                   [df_users, df_sessions, df_subs, df_invoices, df_plans, df_events]):
    if df.empty:
        print(f"⚠️  {name}: ПУСТАЯ ТАБЛИЦА!")
    else:
        print(f"✓ {name}: {len(df)} строк")

✓ Таблица users успешно загружена (2000 строк)
✓ Таблица sessions успешно загружена (5914 строк)
✓ Таблица subscriptions успешно загружена (658 строк)
✓ Таблица invoices успешно загружена (2167 строк)
✓ Таблица plans успешно загружена (3 строк)
✓ Таблица events успешно загружена (16251 строк)
Users: (2000, 3)
Sessions: (5914, 7)
Subscriptions: (658, 5)
Invoices: (2167, 9)
Plans: (3, 4)
Events: (16251, 3)
Размер DataFrame: <class 'tuple'>
Количество строк: 2000
Количество столбцов: 3
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      2000 non-null   int64 
 1   signup_date  2000 non-null   object
 2   region       2000 non-null   object
dtypes: int64(1), object(2)
memory usage: 47.0+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5914 entries, 0 to 5913
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype

,user_id,signup_date,region
0,1,2024-04-12,Казань
1,2,2024-03-15,Москва
2,3,2024-02-22,Новосибирск
3,4,2024-02-27,Казань
4,5,2024-01-15,Москва


,session_id,user_id,session_date,utm_source,utm_medium,utm_campaign,is_first_session
0,sess_1_1,1,2024-04-12,google,cpc,generic,True
1,sess_1_2,1,2024-05-03,google,cpc,generic,False
2,sess_1_3,1,2024-07-04,google,cpc,generic,False
3,sess_2_1,2,2024-03-15,google,cpc,brand,True
4,sess_2_2,2,2024-03-18,google,cpc,brand,False


,subscription_id,user_id,plan_id,start_date,status
0,1001,1,2,2024-04-16,active
1,1002,2,2,2024-03-19,churned
2,1003,5,1,2024-01-26,churned
3,1004,10,1,2024-03-05,churned
4,1005,13,2,2024-04-04,churned


,invoice_id,subscription_id,user_id,period_start,period_end,invoice_date,amount,paid,is_initial
0,50001,1001,1,2024-04-16,2024-05-16,2024-04-16,999.0,True,True
1,50002,1001,1,2024-05-16,2024-06-15,2024-05-16,999.0,True,False
2,50003,1001,1,2024-06-15,2024-07-15,2024-06-15,999.0,True,False
3,50004,1001,1,2024-07-15,2024-08-14,2024-07-15,999.0,True,False
4,50005,1001,1,2024-08-14,2024-09-13,2024-08-14,999.0,True,False


,plan_id,plan_name,period,price
0,1,Basic,monthly,499.0
1,2,Pro,monthly,999.0
2,3,Business,monthly,1999.0


,user_id,event_date,event_name
0,1,2024-04-12,app_open
1,1,2024-04-13,app_open
2,1,2024-04-13,feature_b
3,1,2024-04-15,app_open
4,1,2024-04-17,app_open


ПРОВЕРКА НА ПУСТЫЕ ТАБЛИЦЫ:
══════════════════════════════════════════════════
✓ users: 2000 строк
✓ sessions: 5914 строк
✓ subscriptions: 658 строк
✓ invoices: 2167 строк
✓ plans: 3 строк
✓ events: 16251 строк


In [15]:
# ----------------------
# 3. Создаём staging-таблицы
# ----------------------
from sqlalchemy import types
import numpy as np

# STG_USERS: очистка и нормализация
df_stg_users = df_users.copy()
df_stg_users['signup_date'] = pd.to_datetime(df_stg_users['signup_date'], errors='coerce')
df_stg_users = df_stg_users.drop_duplicates(subset='user_id')
print(f"STG_USERS: {len(df_stg_users)} строк после очистки")

# STG_SESSIONS: проверяем дубликаты, типы данных
df_stg_sessions = df_sessions.copy()
df_stg_sessions['session_date'] = pd.to_datetime(df_stg_sessions['session_date'], errors='coerce')
df_stg_sessions = df_stg_sessions.drop_duplicates(subset='session_id')
print(f"STG_SESSIONS: {len(df_stg_sessions)} строк после очистки")

# STG_SUBSCRIPTIONS: статус и даты
df_stg_subs = df_subs.copy()
df_stg_subs['start_date'] = pd.to_datetime(df_stg_subs['start_date'], errors='coerce')

# Безопасная обработка end_date (если колонка существует)
if 'end_date' in df_stg_subs.columns:
    df_stg_subs['end_date'] = pd.to_datetime(df_stg_subs['end_date'], errors='coerce')

df_stg_subs['status'] = df_stg_subs['status'].fillna('unknown').str.upper().str.strip()
print(f"STG_SUBSCRIPTIONS: {len(df_stg_subs)} строк после очистки")

# STG_INVOICES: даты и суммы
df_stg_invoices = df_invoices.copy()
df_stg_invoices['period_start'] = pd.to_datetime(df_stg_invoices['period_start'], errors='coerce')
df_stg_invoices['period_end'] = pd.to_datetime(df_stg_invoices['period_end'], errors='coerce')
df_stg_invoices['invoice_date'] = pd.to_datetime(df_stg_invoices['invoice_date'], errors='coerce')
df_stg_invoices['amount'] = pd.to_numeric(df_stg_invoices['amount'], errors='coerce')
print(f"STG_INVOICES: {len(df_stg_invoices)} строк после очистки")

# STG_EVENTS: даты и события (исправлено - используем event_name вместо event_type)
df_stg_events = df_events.copy()
df_stg_events['event_date'] = pd.to_datetime(df_stg_events['event_date'], errors='coerce')

# Обрабатываем event_name (а не event_type)
if 'event_name' in df_stg_events.columns:
    df_stg_events['event_name'] = df_stg_events['event_name'].fillna('unknown').str.upper().str.strip()
else:
    print("⚠️  Колонка 'event_name' отсутствует в events")

print(f"STG_EVENTS: {len(df_stg_events)} строк после очистки")

# ----------------------
# 4. Записываем staging-таблицы обратно в БД
# ----------------------
staging_tables = {
    "stg_users": df_stg_users,
    "stg_sessions": df_stg_sessions,
    "stg_subscriptions": df_stg_subs,
    "stg_invoices": df_stg_invoices,
    "stg_events": df_stg_events
}

# Выводим информацию о колонках для отладки
print("\nСтруктура таблиц:")
for table_name, df in staging_tables.items():
    print(f"{table_name}: {list(df.columns)}")

for table_name, df in staging_tables.items():
    try:
        # Сохраняем с обработкой ошибок
        df.to_sql(
            table_name, 
            engine, 
            schema=DB_SCHEMA, 
            if_exists='replace', 
            index=False
        )
        print(f"✓ Таблица {DB_SCHEMA}.{table_name} сохранена ({len(df)} строк)")
        
    except Exception as e:
        print(f"✗ Ошибка сохранения {table_name}: {e}")

print("✓ Все staging-таблицы обработаны и сохранены")

STG_USERS: 2000 строк после очистки
STG_SESSIONS: 5914 строк после очистки
STG_SUBSCRIPTIONS: 658 строк после очистки
STG_INVOICES: 2167 строк после очистки
STG_EVENTS: 16251 строк после очистки

Структура таблиц:
stg_users: ['user_id', 'signup_date', 'region']
stg_sessions: ['session_id', 'user_id', 'session_date', 'utm_source', 'utm_medium', 'utm_campaign', 'is_first_session']
stg_subscriptions: ['subscription_id', 'user_id', 'plan_id', 'start_date', 'status']
stg_invoices: ['invoice_id', 'subscription_id', 'user_id', 'period_start', 'period_end', 'invoice_date', 'amount', 'paid', 'is_initial']
stg_events: ['user_id', 'event_date', 'event_name']
✓ Таблица analytics.stg_users сохранена (2000 строк)
✓ Таблица analytics.stg_sessions сохранена (5914 строк)
✓ Таблица analytics.stg_subscriptions сохранена (658 строк)
✓ Таблица analytics.stg_invoices сохранена (2167 строк)
✓ Таблица analytics.stg_events сохранена (16251 строк)
✓ Все staging-таблицы обработаны и сохранены
